# Snow Density Calculations

This notebook demonstrates methods for calculating density from common snow pit measurements (hand hardness and grain form) using both the local snowpyt_mechparams package and snowpylot for CAAML parsing.


In [4]:
# Import Libraries
import os
import xml.etree.ElementTree as ET
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
import glob

# Import snowpylot for CAAML parsing
from snowpylot import caaml_parser

# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpyt_mechparams import density


Parse Snowpit Files

In [9]:
all_pits = []

folder_path = 'data'

for file in os.listdir(folder_path):
    file_path = folder_path + "/" + file  # create the file path
    all_pits.append(caaml_parser(file_path))


In [ ]:
# Collect relevant data from each snowpit

pit_info = []
layer_info = []

for pit in all_pits:
    pit_dict = {
        'pit_id': pit.core_info.pit_id,
        'layer_count': len(pit.snow_profile.layers),
    }
    pit_info.append(pit_dict)

    for layer in pit.snow_profile.layers:
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'depth_top': layer.depth_top,
            'thickness': layer.thickness,
            'hand_hardness': layer.hardness,
            'wetness': layer.wetness,
            'layer_of_concern': layer.layer_of_concern,
            'grain_form_primary': layer.grain_form_primary,
            'grain_form_secondary': layer.grain_form_secondary,
        }
        layer_info.append(layer_dict)

# Create a dataframe from the pit and layer info
pit_df = pd.DataFrame(pit_info)
layer_df = pd.DataFrame(layer_info)


In [ ]:
# total number of snowpits
len(all_pits)


43829

Implement Geldsetzer Method